Ok here is the gameplane!

1. We will use this model: https://github.com/GouMinghao/rgb_matters/blob/main/rgbd_graspnet/net/fastpose.py
2. Start with small resnet, un intialized weights
3. make sure that loss decreases!
4. need to change dimensions slightly to work for this case...
5. To avoid doing that I can actually perhaps just scale up the env img?... actually smaller image better as less weights!

Ok so the env should have the option to set side_size=10

What will success look like? achieving low loss. but ultimatly we care about success rate.... moving average... 

Generate dataset.... full image for both image and label... Label is always a the tru case of the heatmap, which is 1 or 0 for where grasp would succeed or not.


See design of dataset here: https://github.com/GouMinghao/rgb_matters/blob/main/rgbd_graspnet/data/graspnet.py

Also CIFAR:

https://docs.pytorch.org/vision/main/_modules/torchvision/datasets/cifar.html#CIFAR10

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable
import gymnasium as gym
import bam_gym 
from bam_gym import print_reset, print_step

np.random.seed(1)

In [ ]:
screen_size = 32
max_radius = 3
min_radius = 2
num_circles = 1
render_screen_scale = 10

env = gym.make("bam_local/GraspXY", render_mode="human", screen_size=(screen_size, screen_size), max_radius=max_radius, min_radius=min_radius, num_circles=num_circles, render_screen_scale=render_screen_scale)
print("Action Space:", env.action_space)
print("Observation Space:", env.observation_space)
state, info = env.reset()
print(state.shape)

In [ ]:
state, info = env.reset()

for i in range(10):
    action = env.action_space.sample()
    # action = policy(state)
    state, reward, terminated, truncated, info = env.step(action)
    # print_step(action, state, reward, terminated, truncated, info, i)

env.close()

In [ ]:
env.unwrapped.get_binary_label()
plt.imshow(env.unwrapped.get_binary_label(), cmap='gray', vmin=0, vmax=1)
plt.colorbar()

In [ ]:
env_data_pairs: list[tuple[np.ndarray, np.ndarray]] = env.unwrapped.create_dataset(100)

plt.title("Histogram of image pixels")
plt.hist(env_data_pairs[0][0].flatten(), 255)
plt.show()

In [ ]:
def show_image_grid(image_label_list, nrows=4, ncols=8, figsize=(16, 8), cmap=None, show_labels=False):
    """Display a grid of images (optionally with labels) from a list of (image, label) tuples."""
    total = nrows * ncols
    fig, axes = plt.subplots(nrows, ncols, figsize=figsize)
    axes = axes.flatten()
    for idx, (img, label) in enumerate(image_label_list[:total]):
        ax = axes[idx]
        if img.ndim == 2 or (img.ndim == 3 and img.shape[2] == 1):
            ax.imshow(img.squeeze(), cmap=cmap or 'gray')
        else:
            ax.imshow(img)
        ax.axis('off')
        if show_labels:
            ax.set_title("Label")
            if label is not None:
                if label.ndim == 2:
                    ax.imshow(label, cmap='jet', alpha=0.3)
    for ax in axes[len(image_label_list[:total]):]:
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# Example usage:
show_image_grid(env_data_pairs, nrows=4, ncols=8, show_labels=False)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import make_grid

from torchinfo import summary

In [ ]:
train_dataset_size = 60000
test_dataset_size = 10000



def to_tensor_pairs(numpy_pairs: list[tuple[np.ndarray, np.ndarray]]):
    tensor_pairs = []
    for img, mask in numpy_pairs:
        img_t = torch.from_numpy(img).permute(2,0,1).float() / 255.0
        mask_t = torch.from_numpy(mask).unsqueeze(0).float()
        tensor_pairs.append((img_t, mask_t))
    return tensor_pairs

train_data_pairs = to_tensor_pairs(env.unwrapped.create_dataset(train_dataset_size))

test_data_pairs = to_tensor_pairs(env.unwrapped.create_dataset(test_dataset_size))


print("Size of train tensor dataset:", len(train_data_pairs))
print("Shape of first image:", train_data_pairs[0][0].shape)
print("Shape of first mask:", train_data_pairs[0][1].shape)

print("Size of test tensor dataset:", len(test_data_pairs))
print("Shape of first image:", test_data_pairs[0][0].shape)
print("Shape of first mask:", test_data_pairs[0][1].shape)

In [ ]:
imgs = torch.stack([img for img, _ in train_data_pairs], dim=0)

# Compute per-channel mean and std across N,H,W
mean = imgs.mean(dim=(0,2,3))   # [3]
std  = imgs.std(dim=(0,2,3))    # [3]

print("Mean:", mean)
print("Std:", std)

In [ ]:

class HeatmapDataset(Dataset):
    def __init__(
        self,
        input_label_pairs: list[tuple[torch.Tensor, torch.Tensor]],
        transform: Callable | None = None,
        label_transforms: Callable | None = None,
    ):
        """
        tensor_pairs: list of (img, mask) where
            img: torch.Tensor [3,H,W], float in [0,1]
            mask: torch.Tensor [1,H,W], float {0,1}
        transform: applied to images
        label_transforms: applied to masks
        """
        self.data = input_label_pairs
        self.transform = transform
        self.label_transforms = label_transforms


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img, label = self.data[idx]

        if self.transform:
            img = self.transform(img)
        if self.label_transforms:
            label = self.label_transforms(label)

        return img, label


transform = transforms.Compose([
    # transforms.RandomCrop(32, padding=4),
    # transforms.RandomHorizontalFlip(),
    # transforms.ToTensor(),
    transforms.Normalize(mean, std, inplace=True),
])
label_transforms = None
# transforms.Compose([
#     # transforms.ToTensor(),
#     # transforms.Normalize(mean, std, inplace=True),
# ])


batch_size = 2**9
print(f"Batch size: {batch_size}")


train_dataset = HeatmapDataset(train_data_pairs, transform=transform, label_transforms=label_transforms)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

test_dataset = HeatmapDataset(test_data_pairs, transform=transform, label_transforms=label_transforms)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)


In [ ]:
def show_batch(data_loader: DataLoader):
    for images, labels in data_loader:
        fig, ax = plt.subplots(figsize=(12, 12))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images[:64], nrow=8).permute(1, 2, 0))
        break

show_batch(train_dataloader)

In [ ]:

def show_batch_histograms(data_loader: torch.utils.data.DataLoader, n_batches: int = 1):
    """
    Show histograms of pixel values from a few batches.
    - Plots per-channel histograms for images
    - Plots mask pixel distribution
    """
    for batch_idx, (images, masks) in enumerate(data_loader):
        # Flatten image values: [B, C, H, W] -> [Npixels_per_channel]
        img_vals = images.permute(1,0,2,3).reshape(images.shape[1], -1)  # [C, N]
        
        # Flatten mask values: [B,1,H,W] -> [N]
        mask_vals = masks.reshape(-1)
        
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))

        # --- Image histograms (per channel) ---
        colors = ['r', 'g', 'b'] if images.shape[1] == 3 else ['k']
        for c in range(images.shape[1]):
            axes[0].hist(img_vals[c].numpy().ravel(), bins=50, color=colors[c], alpha=0.5, label=f"Channel {c}")
        axes[0].set_title("Image Pixel Distribution")
        axes[0].legend()

        # --- Mask histogram ---
        axes[1].hist(mask_vals.numpy().ravel(), bins=50, color='gray', alpha=0.7)
        axes[1].set_title("Mask Pixel Distribution")

        plt.show()
        
        if batch_idx + 1 >= n_batches:
            break
show_batch_histograms(train_dataloader, n_batches=2)


In [ ]:
from bam_grasp.net.layers.Heatmap_Resnet import HeatmapResNet

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HeatmapResNet(num_layers=18, out_channels=1, output_downsample=4).to(device)
# print(model)


In [ ]:
summary(
    model, 
    input_size=(batch_size, 3, 32, 32),  # Example input size for CIFAR images
    col_names=["input_size", "output_size", "num_params", "params_percent", "kernel_size", "mult_adds"]
)

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr=0.01)
# scheduler = torch.optim.lr_scheduler.LinearLR(optimizer)



In [ ]:
# Test one batch
img, label = next(iter(train_dataloader))
img, label = img.to(device), label.to(device)
print(img.shape, label.shape)
logits = model(img) 
print(logits.shape)

In [ ]:
train_losses = []
test_losses = []
test_accuracies = []
epoch_total = 0


In [ ]:
def train() -> dict[str, float]:
    model.train()
    n_total = len(train_dataloader.dataset)
    n_batches = len(train_dataloader)
    running_loss = 0.0

    for batch, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X) 
        resized_pred = nn.functional.interpolate(pred, size=y.shape[-2:], mode="nearest")

        train_loss: torch.Tensor = loss_fn(resized_pred, y) 

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        running_loss += train_loss.item()

        if batch % 10 == 0:
            loss, current = train_loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{n_total:>5d}]")

        if batch == 10:
            break # slow down learning a bit so I can see the logs!

    avg_loss = running_loss / n_batches
    return {"avg_loss": avg_loss}


In [ ]:
def test() -> dict[str, float]:
    n_total = len(test_dataloader.dataset)
    n_batches = len(test_dataloader)
    model.eval()
    test_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for X, y in test_dataloader:
            X: torch.Tensor = X.to(device)
            y: torch.Tensor = y.to(device)
            logits: torch.Tensor = model(X)

            # accumulate loss
            test_loss += loss_fn(logits, y).item()

            # BUG: you cannot use this classificaiton style accuracy
            # correct += (logits.argmax(1) == y).type(torch.float).sum().item()

            # --- accuracy ---
            N, C, H, W = logits.shape
            # flatten spatial+channel dims
            logits_flat = logits.view(N, -1)   # (N, C*H*W)
            y_flat = y.view(N, -1)             # (N, H*W) or (N, C*H*W) depending on encoding

            # argmax per sample
            pred_idx = logits_flat.argmax(dim=1)  # (N,)

            hits = y_flat[torch.arange(N), pred_idx]          # (N,)
            correct += (hits == 1).sum().item()
            total += N


    avg_loss = test_loss / n_batches
    accuracy = correct / n_total
    print(correct, n_total)

    print(f"Test Error: \n Accuracy: {(100*accuracy):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    return {"test_loss": test_loss, "test_accuracy": accuracy}


In [ ]:
for epoch in range(10):
    epoch_total += 1
    print(f"Epoch {epoch_total}\n-------------------------------")
    train_info = train()
    test_info = test()

    train_losses.append(train_info["avg_loss"])
    test_losses.append(test_info["test_loss"])
    test_accuracies.append(test_info["test_accuracy"])


In [ ]:
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.plot(train_losses, label="Train Loss")
plt.plot(test_losses, label="Test Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1,2,2)
plt.plot(test_accuracies, label="Test Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.show()

In [ ]:

def preprocess_state(state: np.ndarray):

    transform = transforms.Compose([
        # transforms.RandomCrop(32, padding=4),
        # transforms.RandomHorizontalFlip(),
        # transforms.ToTensor(),
        transforms.Normalize(mean, std, inplace=False),
    ])

    state_tensor = torch.from_numpy(state).permute(2,0,1).float() / 255.0 # Convert to tensor as above
    state_tensor = transform(state_tensor) # Apply same transforms as test dataset
    state_tensor = state_tensor.unsqueeze(0) # add batch dim
    state_tensor = state_tensor.to(device) # move to same device as model

    return state_tensor

class HeatmapPolicy():
    def __init__(self, model: nn.Module, preprocess_state: Callable, epsilon=0.2):

        self.model = model
        self.model.eval()
        self.epsilon = epsilon
        self.iter = 0
        self.preprocess_state = preprocess_state
        self.last_heatmaps = None


    def __call__(self, state: np.ndarray):
        self.iter += 1

        state = self.preprocess_state(state)


        heatmaps = self.model(state)
        self.last_heatmaps = heatmaps
        B, C, H, W = heatmaps.shape

        if np.random.random() <= self.epsilon:
            cs = torch.randint(C, (B,)) 
            xs = torch.randint(W, (B,))
            ys = torch.randint(H, (B,))

            coords = torch.stack([xs, ys, cs], dim=1)  # [B,3]
            if C == 1:
                coords = coords[:, :2]  # drop channel column -> [B,2]

            if B == 1:
                return tuple(int(v) for v in coords[0])
            return coords

        # Exploit: global argmax over (C,H,W) for each sample
        heatmaps_flat = heatmaps.view(B, -1)                     # [B, C*H*W]
        argmax_idx = heatmaps_flat.argmax(dim=1)               # [B]

        # Decode to (c, y, x), then output as (c, x, y) (or (x,y) if C==1)
        c = argmax_idx // (H * W)                     # [B]
        rem = argmax_idx % (H * W)
        y = rem // W
        x = rem % W

        if C == 1:
            if B == 1:
                return int(x.item()), int(y.item())
            return torch.stack([x, y], dim=1)         # [B,2] -> (x,y)
        else:
            if B == 1:
                return int(c.item()), int(x.item()), int(y.item())
            return torch.stack([x, y, c], dim=1)      # [B,3] -> (c,x,y)

print(preprocess_state(state).shape)

policy = HeatmapPolicy(model, preprocess_state, epsilon=0.2)

In [ ]:
state, info = env.reset()
action = policy(state)

def display_heatmap(state, action, policy):


    plt.title("state")
    plt.imshow(state)
    plt.show()


    plt.title("Heatmap")
    plt.imshow(policy.last_heatmaps[0,0,:,:].detach().cpu().numpy())
    plt.colorbar()

    x, y = action if isinstance(action, (tuple, list)) and len(action) == 2 else (action[1], action[2])
    plt.scatter([x], [y], color='red', marker='x', s=100, label='Action')
    plt.legend()
    plt.show()

display_heatmap(state, action, policy)

In [ ]:
reward_history = []
epsilon_history = []

In [ ]:
env.unwrapped.render_mode = "none"

In [ ]:
state, info = env.reset()

policy.epsilon = 0.0

for i in range(100):
    # action = env.action_space.sample()
    action = policy(state)
    new_state, reward, terminated, truncated, info = env.step(action)
    reward_history.append(reward)
    epsilon_history.append(policy.epsilon)

    if reward == 0:
        display_heatmap(state, action, policy)
    state = new_state

    # print_step(action, state, reward, terminated, truncated, info, i)

env.close()

In [ ]:
plt.plot(reward_history, label="Reward")
plt.plot(epsilon_history, label="Epsilon", color='red')
# Compute moving average
window_size = 50
if len(reward_history) >= window_size:
    moving_avg = np.convolve(reward_history, np.ones(window_size)/window_size, mode='valid')
    plt.plot(range(window_size-1, len(reward_history)), moving_avg, label=f"Moving Avg ({window_size})", color='orange')

plt.xlabel("Episode")
plt.ylabel("Reward")
plt.title("Reward History and Moving Average")
plt.legend()
plt.show()